In [1]:
import pywatershed
import pandas as pd
from pathlib import Path as pl
import json
import numpy as np
from pywatershed.utils.parameters import JSONParameterEncoder
import sys
sys.path.append('../scripts/')
from pest_utils import pars_to_tpl_entries

In [2]:
def write_to_json_tpl(pars, json_filename):
    with open(json_filename, "w") as ofp:
        ofp.write('ptf ~\n')
        json.dump(
            pars,
            ofp,
            indent=4,
            cls=JSONParameterEncoder,
        )
    # this sucks - should be a more direct way but whatevs. it verks
    inlines = open(json_filename, 'r').readlines()
    with open(json_filename, 'w') as ofp:
        [ofp.write(i.replace('"~','~').replace('~"','~')) for i in inlines]

## we will be running on each cutout eventually, but start with one

In [3]:
wkdir = pl('../NHM_extractions/20230110_pois_haj/01473000/')

In [4]:
# read in the oldskool parameter file as nested dictionaries (whatup json?)
pardat = pywatershed.parameters.PrmsParameters.load(wkdir / "myparam.param")

In [5]:
pars = pardat.parameters

In [6]:
pars['nhm_id']

array([5621, 5625, 5628, 5635, 5637, 5643, 5678, 5679, 5686, 5690, 5693,
       5697, 5703, 5728, 7128, 7156, 7157])

## create a template file version of json-style input

In [7]:
pars

{'nhm_id': array([5621, 5625, 5628, 5635, 5637, 5643, 5678, 5679, 5686, 5690, 5693,
        5697, 5703, 5728, 7128, 7156, 7157]),
 'nhm_seg': array([2308, 2309, 2310, 2289, 2290, 2292, 2294, 2300, 2302, 2303]),
 'poi_gage_id': array(['01472198', '01472199', '01472810', '01472620', '01473000'],
       dtype=object),
 'K_coef': array([ 1.758604,  4.709235,  1.763242, 17.503676, 19.530437, 22.085455,
         4.1999  ,  6.890914, 16.229709,  2.078517]),
 'adjmix_rain': array([[0.668501, 0.668501, 0.668501, 0.668501, 0.668501, 0.668501,
         0.668501, 0.668501, 0.668501, 0.668501, 0.668501, 0.668501,
         0.668501, 0.668501, 0.668501, 0.668501, 0.668501],
        [0.668501, 0.668501, 0.668501, 0.668501, 0.668501, 0.668501,
         0.668501, 0.668501, 0.668501, 0.668501, 0.668501, 0.668501,
         0.668501, 0.668501, 0.668501, 0.668501, 0.668501],
        [0.668501, 0.668501, 0.668501, 0.668501, 0.668501, 0.668501,
         0.668501, 0.668501, 0.668501, 0.668501, 0.668501, 0.6685

In [8]:
# find list of hrus
hrus = pars['nhm_id']
hrus

array([5621, 5625, 5628, 5635, 5637, 5643, 5678, 5679, 5686, 5690, 5693,
       5697, 5703, 5728, 7128, 7156, 7157])

In [9]:
segs = pars['nhm_seg']
segs

array([2308, 2309, 2310, 2289, 2290, 2292, 2294, 2300, 2302, 2303])

## run through the parameters

In [10]:
par_starting_vals = pd.DataFrame(columns=['parname','parval1'])

In [11]:
par_starting_vals

,parname,parval1


### run through all the currently-defined parameters 

In [12]:
par_starting_vals = pars_to_tpl_entries(pars, 'adjmix_rain', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'carea_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'cecn_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'emis_noppt', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'fastcoef_lin', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'freeh2o_cap', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'gwflow_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'jh_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'mann_n', hrus, segs, par_starting_vals, hru_based=False, 
                    seg_based=True, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'potet_sublim', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'rad_trncf', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'radmax', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'rain_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'slowcoef_sq', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'smidx_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'snarea_thresh', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'snowinfil_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'snow_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'soil2gw_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'soil_moist_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'soil_rechr_max_frac', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'ssr2gw_exp', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'ssr2gw_rate', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'tmax_allrain_offset', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'tmax_allsnow', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'tmax_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'tmin_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)


In [13]:
par_starting_vals

,parname,parval1
0,adjmix_rain:hru_5621:mon_1,0.668501
1,adjmix_rain:hru_5625:mon_1,0.668501
2,adjmix_rain:hru_5628:mon_1,0.668501
3,adjmix_rain:hru_5635:mon_1,0.668501
4,adjmix_rain:hru_5637:mon_1,0.668501
...,...,...
199,tmin_cbh_adj:hru_5703:mon_12,1.383938
200,tmin_cbh_adj:hru_5728:mon_12,3.000000
201,tmin_cbh_adj:hru_7128:mon_12,0.691790
202,tmin_cbh_adj:hru_7156:mon_12,2.468683


### now run through all the other input parameters....

### once we have all the parameter arrays replaced by names, we can write out the template file

In [14]:
write_to_json_tpl(pars, wkdir / 'test.tpl')
par_starting_vals.to_csv(wkdir / 'starting_par_vals.dat', index=None, sep=' ')